In [1]:
import os

In [2]:
%pwd

'c:\\Users\\amitt\\Documents\\DataScienceProject\\research'

In [3]:
os.chdir("../")

In [4]:
import pandas as pd
data = pd.read_csv(r"artifacts\data_ingestion\winequality-red.csv")

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [6]:
# checking for null values
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [7]:
data.shape

(1599, 12)

In [8]:
from dataclasses import dataclass
from pathlib import Path
import urllib.request as request
from src.datascience import logger
import zipfile

@dataclass
class DataValidationconfig:

    root_dir : Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict


In [9]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directory

In [10]:
# updating the configuration manager
class ConfigurationManager:

    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 param_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMAS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_file_path)
        self.schema = read_yaml(schema_file_path)


        create_directory([self.config.artifacts_root])


    def get_data_validation_config(self) -> DataValidationconfig:


        config = self.config.data_validation
        # we are getting all the columns from the schema file
        schema = self.schema.COLUMNS

        create_directory([config.root_dir])

        data_validation_config = DataValidationconfig    (


            root_dir=Path(config.root_dir),
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir =Path(config.unzip_data_dir),
            all_schema = schema

        )

        return data_validation_config


In [11]:
import os
from src.datascience import logger

In [12]:
class DataValidation:

    def __init__(self,config:DataValidationconfig):

        self.config = config

    # downloading the zip file
    def validate_all_columns(self) -> bool:

        try:

            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)

            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            all_data_types = self.config.all_schema.values()


            print("all schemas",self.config.all_schema)

            # verifying the schema is correct or not with all columns
            for col in all_cols:

                if col not in all_schema and col.dtype != self.config.all_schema[col]:

                    validation_status = False

                    with open(self.config.STATUS_FILE,"w") as f:

                        f.write(f"failed {validation_status}" )

                else:
                    validation_status = True

                    with open(self.config.STATUS_FILE,"w") as f:

                        f.write(f"validation status {validation_status}" )

            return validation_status 
        
        except Exception as e:

            raise e

In [13]:
try:
    config = ConfigurationManager()

    data_validation_config = config.get_data_validation_config()

    data_validation = DataValidation(data_validation_config)

    data_validation.validate_all_columns()
except Exception as e:

    raise e

[2024-11-01 18:31:27,571 - INFO - common - yaml file : config\config.yaml loaded successfully]
the content is : {content}, for yaml file : {yaml_file}
[2024-11-01 18:31:27,571 - INFO - common - yaml file : params.yaml loaded successfully]
the content is : {content}, for yaml file : {yaml_file}
[2024-11-01 18:31:27,578 - INFO - common - yaml file : schema.yaml loaded successfully]
the content is : {content}, for yaml file : {yaml_file}
[2024-11-01 18:31:27,579 - INFO - common - directory : artifacts created successfully]
[2024-11-01 18:31:27,580 - INFO - common - directory : artifacts/data_validation created successfully]
all schemas {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}
